In [52]:
import numpy as np
from gurobipy import *

In [11]:
def readFile(nom_file):
    nom_file+=".txt"
    #file = open(nom_file,"r")
    with open(nom_file, 'r') as f:
        l = [[int(num) for num in line.split()] for line in f]
        #print(l)
    return np.asarray(l)
    f.close()

### les données
les données sont un ensemble d'alternatives évaluées selon n caractères

In [14]:
matrice=readFile("exempleinstance")
print(matrice)
print(matrice.shape)
#matrice a un type de list

[[32 42 26]
 [26 45 35]
 [39 43 18]
 [45 23 36]
 [37 27 37]]
(5, 3)


In [17]:
"""
matrice: taille m*n
         m est le nombre des solution
         n le nombre de criteres
         les donnees enregistres dans le ficher txt
matrice_pmr: talle n*n
         matrice enregistre pmr pour toutes les pairs

mr:taille n*1
"""

'\nmatrice: taille m*n\n         m est le nombre des solution\n         n le nombre de criteres\n         les donnees enregistres dans le ficher txt\nmatrice_pmr: talle n*n\n         matrice enregistre pmr pour toutes les pairs\n\nmr:taille n*1\n'

### PMR
the PMR is the worst-case when recommending solution x instead of x' to the DM
the PMR matrix here contains all possible pairs of alternatives. The pmr in the diagonal is 0.

In [5]:
def pmr(matrice):    
    nb_alt,nb_citere=matrice.shape
    matrice_pmr=np.zeros((nb_alt,nb_alt))
    for i in range(nb_alt):
        for j in range(nb_alt):
            if i!=j:               
                cpt=np.ndarray.max(np.array(matrice[i,:]-matrice[j,:]))
                matrice_pmr[i,j]=cpt
    return matrice_pmr

In [15]:
pmr_matrice=pmr(matrice)
print(pmr_matrice)

[[ 0.  6.  8. 19. 15.]
 [ 9.  0. 17. 22. 18.]
 [ 7. 13.  0. 20. 16.]
 [13. 19. 18.  0.  8.]
 [11. 11. 19.  4.  0.]]


### Max regret(MR)
the MR represente the worst-case loss when choosing solution x instead of any other feasible solution. For exemple, the worst-case lost case is when the true solution is solution4 but we have solution1 instead. And there is always one MR for each solution.

In [16]:
def max_regret(pmrM):
    nb_alt=pmrM.shape[0]
    max_regret=np.zeros(nb_alt)
    for i in range(nb_alt):
        max_regret[i]=np.ndarray.max(pmrM[i,:])
    return max_regret

In [19]:
mr=max_regret(pmr_matrice)
print(mr)

[19. 22. 20. 19. 19.]


### le poids caches
le poids caches ici est pour simuler un utilisateur et sa reponse.

In [22]:
poid=np.array([0.326851,0.318176,0.354973])
print(poid)

[0.326851 0.318176 0.354973]


In [76]:
def Query(poid,sol1,sol2):
    #print(poid*sol1,poid*sol2)
    print(np.sum(poid*sol1),np.sum(poid*sol2))
    if(np.sum(poid*sol1)>np.sum(poid*sol2)):
        
        return False
    else:
        return True

In [91]:
def expression(q,ligne):
    expr=LinExpr()
    for i in range(len(q)):
        #print(i,q[i],ligne[i])
        expr+=q[i]*ligne[i]
    #print(expr)
    return expr

In [110]:
def newPMR(m,matrice,pmr,delete,q):
    matrice=np.delete(matrice,delete,0)
    pmr=np.delete(pmr,delete,0)
    pmr=np.delete(pmr,delete,1)
    print(pmr)
    a,b=pmr.shape
    for i in range(a):
        for j in range(b):
            if(i!=j):
                expr1=expression(q,matrice[i,:])
                expr2=expression(q,matrice[j,:])
                m.setObjective(expr1-expr2,GRB.MAXIMIZE)
                m.optimize()
                pmr[i,j]=m.objVal
    print(pmr)
    return matrice,pmr

### MMR
the MiniMax Regret : progressively ask preference queries to the DM until the MMR drops below a given threshold.

In [118]:
#on simule la premiere iteration
def MMR(matrice,poid):
    #initialisation du modele gurobi
    m = Model("s0")
    n=matrice.shape[1]
    q=[]
    for j in range(n):
        q.append(m.addVar(vtype=GRB.CONTINUOUS, name="w%d"%(j+1)))
    #print(type(q[1]))
    expr=LinExpr()
    #la premiere constrainte : la somme est egale a 1
    for j in range(n):
        expr+=q[j]
    m.addConstr(expr==1)
    
    #initialisation
    i=0
    print("iteration",i+1)
    pmr_matrice=pmr(matrice)
    mr=max_regret(pmr_matrice)
    sol=np.argmin(mr)
    print("the solution de min-max regret est solution",sol+1)
    pire_ad=np.argmax(pmr_matrice[sol,:])
    print("son pire adversaire est la solution",pire_ad+1)
    
    expr1=expression(q,matrice[sol,:])
    expr2=expression(q,matrice[pire_ad,:])
    
    #ajouter la nouvelle constrainte
    m.setObjective(expr1-expr2,GRB.MAXIMIZE)
    preference=-1
    if(Query(poid,matrice[sol,:],matrice[pire_ad,:])):
        print("notre utilisateur prefere solution",sol+1)
        m.addConstr(expr1<=expr2)
        delete=pire_ad
        
    else:
        print("notre utilisateur prefere solution",pire_ad+1)
        m.addConstr(expr2<expr1)
        preference=pire_ad
        delete=sol
    
    #resoudre le PL et mettre a jour les matrice
    newPMR(m,matrice,pmr_matrice,delete,q)

In [119]:
MMR(matrice,poid)

iteration 1
the solution de min-max regret est solution 1
son pire adversaire est la solution 4
33.051922000000005 34.805371
notre utilisateur prefere solution 1
[[ 0.  6.  8. 15.]
 [ 9.  0. 17. 18.]
 [ 7. 13.  0. 16.]
 [11. 11. 19.  0.]]
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xf2456d39
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.04s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds
Optimal objective  6.000000000e+00
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00

In [128]:
def MMR2(matrice,poid):
    #initialisation
    ite=0
    m = Model("s0")
    n=matrice.shape[1]
    q=[]
    for j in range(n):
        q.append(m.addVar(vtype=GRB.CONTINUOUS, name="w%d"%(j+1)))
    #print(type(q[1]))
    expr=LinExpr()
    #la premiere constrainte : la somme est egale a 1
    for j in range(n):
        expr+=q[j]
    m.addConstr(expr==1)
    pmr_matrice=pmr(matrice)
    
    while(matrice.shape[0]>1):
        print("c'est iteration",ite+1)
        mr=max_regret(pmr_matrice)
        print(mr)
        sol=np.argmin(mr)
        print("the solution de min-max regret est solution",sol+1)
        pire_ad=np.argmax(pmr_matrice[sol,:])
        print("son pire adversaire est la solution",pire_ad+1)

        expr1=expression(q,matrice[sol,:])
        expr2=expression(q,matrice[pire_ad,:])

        #ajouter la nouvelle constrainte
        m.setObjective(expr1-expr2,GRB.MAXIMIZE)
        preference=-1
        if(Query(poid,matrice[sol,:],matrice[pire_ad,:])):
            print("notre utilisateur prefere solution",sol+1)
            m.addConstr(expr1<=expr2)
            delete=pire_ad

        else:
            print("notre utilisateur prefere solution",pire_ad+1)
            m.addConstr(expr2<=expr1)
            preference=pire_ad
            delete=sol

        #resoudre le PL et mettre a jour les matrice
        matrice,pmr_matrice=newPMR(m,matrice,pmr_matrice,delete,q)
        ite+=1

In [129]:
MMR2(matrice,poid)

c'est iteration 1
[19. 22. 20. 19. 19.]
the solution de min-max regret est solution 1
son pire adversaire est la solution 4
33.051922000000005 34.805371
notre utilisateur prefere solution 1
[[ 0.  6.  8. 15.]
 [ 9.  0. 17. 18.]
 [ 7. 13.  0. 16.]
 [11. 11. 19.  0.]]
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xf2456d39
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  6.000000000e+00
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Coefficient statistic

Optimize a model with 3 rows, 3 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5775862e+31   3.094828e+30   1.577586e+01      0s
       2   -1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.05 seconds
Optimal objective -1.000000000e+00
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 3 rows, 3 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.6000000e+30   1.462500e+30   5.600000e+00      0s
       1    6.3636364e-01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.04 seconds
Optimal object